# ✅ LCEL + PDF RAG + Gradio 챗봇 (NVIDIA API 사용)

이 노트북은 **NVIDIA API(OpenAI 호환 엔드포인트)**를 사용하여 LLM을 호출하고, LangChain의 **LCEL**로 RAG 파이프라인을 구성한 뒤, **Gradio**로 간단한 챗봇 UI를 만드는 예제입니다.

구성:
1) **LCEL 기본 예제** (RAG에 필요한 수준만)  
2) **PDF 로딩 → 청크 분할 → 임베딩 → 벡터스토어(FAISS)**  
3) **LLM 단독 Gradio 챗봇** (NVIDIA API, Mixtral 8x7B)  
4) **RAG 기반 Gradio 챗봇** (PDF 지식 기반)  

> 모든 설명/프롬프트는 한국어입니다.

## 0) 환경 설정 및 설치

In [1]:
# ⚙️ 필수 라이브러리 설치 (이미 설치되어 있다면 건너뛰어도 됩니다)
# LangChain 최신 버전에서는 커뮤니티 컴포넌트가 분리되어 있어 아래 패키지를 함께 설치합니다.
# %pip install -U langchain langchain-openai langchain-community langchain-huggingface                     sentence-transformers faiss-cpu PyPDF2 gradio requests

## 1) NVIDIA API 키 설정 및 스트리밍 호출 데모

- NVIDIA API는 OpenAI 호환 **Chat Completions** 엔드포인트를 제공합니다.  
- 아래 코드는 **스트리밍**으로 토큰을 받아 출력하는 예시입니다.  
- 모델: `mistralai/mixtral-8x7b-instruct-v0.1`
- API 키는 다음 링크에서 받을 수 있습니다. [https://build.nvidia.com/]

In [11]:
import os, json, requests
from getpass import getpass

#os.environ["NVIDIA_API_KEY"] = "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"

# 🔑 API 키 설정 (환경변수에 저장). 키가 없으면 입력창이 뜹니다.
if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    os.environ["NVIDIA_API_KEY"] = getpass("NVIDIA_API_KEY를 입력하세요 (nvapi-...): ")

invoke_url = "https://integrate.api.nvidia.com/v1/chat/completions"
headers = {
    "accept": "text/event-stream",
    "content-type": "application/json",
    "Authorization": f"Bearer {os.environ['NVIDIA_API_KEY']}",
}

payload = {
    "model": "mistralai/mixtral-8x7b-instruct-v0.1",
    "messages": [{"role":"user","content":"안녕? 한국어로 인사해줘"}],
    "temperature": 0.5,
    "top_p": 1,
    "max_tokens": 256,
    "stream": True
}

def get_stream_token(entry: bytes):
    """스트리밍 응답에서 토큰 텍스트만 추출"""
    if not entry: return ""
    entry = entry.decode("utf-8")
    if entry.startswith("data: "):
        try:
            entry = json.loads(entry[6:])
        except ValueError:
            return ""
    return entry.get("choices", [{}])[0].get("delta", {}).get("content") or ""

# 🚀 스트리밍 호출
response = requests.post(invoke_url, headers=headers, json=payload, stream=True)
try:
    response.raise_for_status()
except Exception as e:
    print("에러 응답:", response.text)
    raise e

print("▶ 모델 응답(스트리밍):", end=" ")
for line in response.iter_lines():
    print(get_stream_token(line), end="")
print("\n")

▶ 모델 응답(스트리밍): 안녕하세요! 한국어로 인사해 주셔서 감사합니다. (Hello! Thank you for greeting me in Korean.)



## 2) LCEL 기본 예제 (RAG에 필요한 요소 위주)

여기서는 LCEL의 핵심인 `RunnableLambda`, `RunnableParallel`, `RunnablePassthrough` 정도만 사용합니다.  
- **목표**: 입력 질의를 받아 전처리 → (병렬로) 원본 보존 및 추가 처리 → 후처리 구조 익히기

In [3]:
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough

def normalize_question(q: str) -> str:
    # 간단 전처리: 좌우 공백 제거 + 마침표 보정
    q = (q or "").strip()
    if q and q[-1] not in ".?!":
        q += "?"  # 질문 형태로 정규화
    return q

def summarize_hint(q: str) -> str:
    # 매우 단순한 요약 힌트 (실전에서는 LLM/규칙기반 활용)
    kw = q[:20]
    return f"요약힌트:{kw}..."

node_normalize = RunnableLambda(normalize_question)

parallel = RunnableParallel({
    "original": RunnablePassthrough(),
    "normalized": node_normalize,
    "hint": RunnableLambda(summarize_hint),
})

print(parallel.invoke("RAG란 무엇인가"))

{'original': 'RAG란 무엇인가', 'normalized': 'RAG란 무엇인가?', 'hint': '요약힌트:RAG란 무엇인가...'}


## 3) PDF 로딩 → 분할 → 임베딩 → FAISS 벡터스토어

- **DocumentLoader**: `PyPDFLoader` (LangChain Community)  
- **텍스트 분할**: `RecursiveCharacterTextSplitter`  
- **임베딩**: `sentence-transformers/all-MiniLM-L6-v2` (로컬)  
- **벡터스토어**: `FAISS`

> `example.pdf` 파일이 현재 작업 디렉터리에 없다면, 데모용 간단 PDF를 자동 생성합니다.

In [4]:
import os

# 데모용 PDF 자동 생성 (없을 때만)
pdf_path = "example.pdf"
if not os.path.exists(pdf_path):
    from reportlab.pdfgen import canvas
    from reportlab.lib.pagesizes import A4
    c = canvas.Canvas(pdf_path, pagesize=A4)
    text = c.beginText(40, 800)
    text.textLines("""
이 문서는 RAG 데모를 위해 자동 생성된 PDF입니다.
RAG(Retrieval-Augmented Generation)는 검색과 생성 모델을 결합하여,
질의와 관련된 문서 조각을 먼저 검색하고, 해당 컨텍스트를 바탕으로 답변을 생성하는 방식입니다.

여기에 임의의 예시 내용을 더 추가합니다.
- LCEL은 LangChain에서 체인을 구성하기 위한 표현 언어입니다.
- RunnableLambda, RunnableParallel, RunnablePassthrough 등을 이용해 파이프라인을 유연하게 설계할 수 있습니다.
- 이 문서의 목적은 벡터스토어 구축과 RAG 챗봇의 동작을 보여주는 것입니다.
""")
    c.drawText(text)
    c.showPage()
    c.save()
    print("데모용 example.pdf를 생성했습니다.")
else:
    print("example.pdf 파일이 이미 존재합니다.")

example.pdf 파일이 이미 존재합니다.


In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("example.pdf")
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=120,
    separators=["\n\n", "\n", " ", ""]
)
chunks = splitter.split_documents(documents)

print(f"페이지 수: {len(documents)}, 청크 수: {len(chunks)}")
print("첫 청크 미리보기:\n", chunks[0].page_content[:200].replace("\n"," "))

페이지 수: 6, 청크 수: 11
첫 청크 미리보기:
 RAG가 해결하는 문제언어모델(LLM)의 한계: 파라미터 안에 든 지식은 최신성이 떨어지고, 출처를 제시하기 어렵고, 길게 기억하지도 못해요.RAG의 아이디어: “답을 만들기 전에 외부 지식베이스에서 관련 문서를 찾아 컨텍스트로 넣어준다.”→ 최신성, 출처 제시, 도메인 특화 정확도가 크게 좋아지고, 파라미터 재학습 없이 지식만 교체·증분 추가 가능.큰 그


In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, emb)

# 간단 검색 테스트
docs = vectorstore.similarity_search("RAG의 목적은 무엇인가요?", k=2)
for i, d in enumerate(docs, 1):
    print(f"Top {i}:", d.page_content[:120].replace("\n"," "),"...")

Top 1: 메모리 RAG(대화 장기 컨텍스트)세션 메모리(요약) + 개인 노트(보안 영역) + 문서 RAG 결합10. 실패 모드 & 디버깅 체크리스트증상 → 원인 → 처방 빠르게 매칭하기관련 문서 못 찾음 → 분할 과도/메타데 ...
Top 2: RAG가 해결하는 문제언어모델(LLM)의 한계: 파라미터 안에 든 지식은 최신성이 떨어지고, 출처를 제시하기 어렵고, 길게 기억하지도 못해요.RAG의 아이디어: “답을 만들기 전에 외부 지식베이스에서 관련 문서를 찾 ...


## 4) NVIDIA API를 LangChain LLM으로 사용하기

`langchain-openai`의 `ChatOpenAI`를 사용하되, **base_url**을 NVIDIA로 설정하면 OpenAI 호환 방식으로 호출할 수 있습니다.

In [7]:
import os
from langchain_openai import ChatOpenAI

# 환경변수 NVIDIA_API_KEY가 이미 설정되어 있어야 합니다.
assert os.environ.get("NVIDIA_API_KEY","").startswith("nvapi-"), "NVIDIA_API_KEY가 설정되지 않았습니다."

llm = ChatOpenAI(
    temperature=0.3,
    model="mistralai/mixtral-8x7b-instruct-v0.1",
    api_key=os.environ["NVIDIA_API_KEY"],
    base_url="https://integrate.api.nvidia.com/v1",
)

# 간단 호출
resp = llm.invoke("한 줄 한국어 인사말을 정중하게 작성해 주세요.")
print(resp.content)

안녕하세요, 귀하의 일을 도와드리기 위해 최선을 다하겠습니다. (Hello, I will do my best to assist you.)


## 5) LCEL로 RAG 체인 구성 (Retriever → Prompt 포맷 → LLM)

- `vectorstore.as_retriever(k=3)` 로 검색 노드 구성  
- `RunnableParallel`로 질문/컨텍스트 병렬 처리  
- 사용자 정의 포맷터(`RunnableLambda`)로 프롬프트 생성  
- `ChatOpenAI` LLM에 전달 → 답변

In [8]:
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnablePassthrough
from langchain.schema import Document

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

def format_prompt(inputs: dict) -> str:
    docs = inputs["context"]
    q = inputs["question"]
    ctx_text = "\n\n".join([d.page_content for d in docs])
    prompt = (
        "다음 문서 컨텍스트를 바탕으로 한국어로 간결하고 정확하게 답하십시오.\n"
        "컨텍스트에서 찾을 수 없는 내용은 추론하지 말고 '문서에서 확인할 수 없습니다.'라고 답하세요.\n"
        "=== 컨텍스트 시작 ===\n"
        f"{ctx_text}\n"
        "=== 컨텍스트 끝 ===\n"
        f"질문: {q}\n"
        "답변:"
    )
    return prompt

rag_node = RunnableParallel({
    "context": retriever,
    "question": RunnablePassthrough(),
}) | RunnableLambda(format_prompt) | llm

# 테스트
print(rag_node.invoke("이 문서의 주제와 RAG의 핵심 과정을 설명해 주세요.").content)

이 문서의 주제는 "Retrieval-Augmented Generation (RAG)"이다. RAG는 언어모델(LLM)의 한계를 극복하기 위해 제안된 기술로, 파라미터 안에 든 지식보다는 외부 지식베이스에서 관련 문서를 찾아 컨텍스트로 활용하여 정확도, 최신성, 출처 제시 측면에서 큰 이점을 가져옵니다.

RAG의 핵심 과정은 다음과 같습니다.

1. 지식 수집 및 정제: PDF, DOCX, 웹 등에서 문서를 수집하고, 전처리 과정을 거칩니다. 이 과정에서 문서는 클린, 분할, 메타데이터 처리가 이루어집니다.
2. 인덱싱: 문서를 벡터화하여 벡터 DB 또는 색인에 저장합니다.
3. 질의 처리: 사용자의 질문을 임베딩화하고, 검색과 재순위화를 수행합니다.
4. 컨텍스트 구성: 중요한 조각을 선택하여 길이 제한 안에서 압축합니다.
5. 답변 생성: LLM 프롬프트에 컨텍스트와 지시문을 넣어 답변을 생성하며, 인용과 출처를 포함시킵니다.
6. 평가 및 로깅: 정확성과 충실성, 비용, 지연 시간, 관측 및 모니터링, 캐싱 및 업데이트 등을 평가합니다.

RAG는 다양한 형태로 발전하고 있으며, 멀티모달, 고급 검색, 그래프 기반 연결 형태 질의, 에이전틱 기능 등을 지원하고 있습니다.


## 6) Gradio: LLM 단독 챗봇

간단한 입력/출력 챗봇 UI. (RAG 미적용)

In [9]:
import gradio as gr

def chat_plain(user_input: str) -> str:
    if not user_input or not user_input.strip():
        return "질문을 입력해 주세요."
    out = llm.invoke(
        f"아래 사용자의 발화를 정중한 한국어로 응답하세요.\n사용자: {user_input}\n응답:"
    )
    return out.content

demo_plain = gr.Interface(
    fn=chat_plain,
    inputs=gr.Textbox(label="질문 입력", placeholder="여기에 질문을 입력하세요"),
    outputs=gr.Textbox(label="모델 응답"),
    title="LLM 단독 챗봇 (NVIDIA Mixtral)",
    description="간단한 LLM 기반 챗봇 데모입니다."
)
# 주피터에서 실행 시 별도의 외부 링크가 표시됩니다.
#demo_plain.launch()  # 필요 시 수동 실행

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## 7) Gradio: PDF 기반 RAG 챗봇

- 사용자의 질문 → 벡터스토어 검색 → 프롬프트 생성 → LLM 호출 → 답변  
- 간단한 1턴 QA 형태 (대화 히스토리 미보존)

In [10]:
def chat_rag(user_input: str) -> str:
    if not user_input or not user_input.strip():
        return "질문을 입력해 주세요."
    try:
        res = rag_node.invoke(user_input)
        # ChatOpenAI는 AIMessage를 반환 → .content
        return getattr(res, "content", str(res))
    except Exception as e:
        return f"오류가 발생했습니다: {e}"

demo_rag = gr.Interface(
    fn=chat_rag,
    inputs=gr.Textbox(label="질문 입력", placeholder="PDF 내용에 대해 질문하세요"),
    outputs=gr.Textbox(label="RAG 응답"),
    title="PDF 기반 RAG 챗봇",
    description="example.pdf 내용을 바탕으로 답변합니다."
)
#demo_rag.launch()  # 필요 시 수동 실행

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


---

### ✅ 팁 / 다음 단계
- **PDF 교체**: `example.pdf` 대신 실제 PDF 파일을 같은 폴더에 두고 파일명을 바꾸세요.
- **스트리밍 UI**: Gradio에서 스트리밍을 구현하려면 `yield`를 사용한 제너레이터 함수를 활용하세요.
- **대화 히스토리**: 간단히는 textbox에 누적해 넘기거나, LangChain의 `MessagesPlaceholder`를 사용해 확장 가능합니다.
- **임베딩 모델 교체**: 한국어 성능을 높이려면 한국어 특화 임베딩 모델로 교체하세요. (예: `jhgan/ko-sroberta-multitask` 등)
- **보안**: 노트북 공유 시 API 키가 노출되지 않도록 주의하세요.

필요하면 `demo_plain.launch()` 또는 `demo_rag.launch()` 셀을 실행하여 UI를 띄우세요.